In [1]:
import os
import pandas as pd
import geopandas as gpd
import numpy as np
from sklearn.cluster import DBSCAN
import optuna
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix

# 1. Load relevant functions

## 1.1 Load the function to group the DataFrame into fire events

In [2]:
def group_into_fire_events(df):
    """
    This function takes in a DataFrame and returns a copy of the DataFrame with three additional columns:
    'date_cluster', 'regional_cluster', and 'event_id'.
    
    Args:
        df (pd.DataFrame): The input DataFrame
        
    Returns:
        df (pd.DataFrame): The output DataFrame with three additional columns: 'date_cluster', 'regional_cluster', and 'event_id'.
    """
    
    df = df.copy()
    df.rename(columns={'latitude_left': 'latitude', 'longitude_left': 'longitude'}, inplace=True)
    
    # Convert 'acq_date' to datetime
    df['date'] = pd.to_datetime(df['date'])
    
    df = df.sort_values(by='date')
    
    # Create 'date_cluster' column based on consecutive dates
    df['date_cluster'] = (df['date'].diff().dt.days > 1).cumsum()
    
    # Function to apply DBSCAN on each consecutive date cluster
    def apply_dbscan(group):
        coords = group[['latitude', 'longitude']].values 
        # Apply DBSCAN, hyperparameters are the same as Climada
        db = DBSCAN(eps=15/111.12, min_samples=1).fit(coords)
        group['regional_cluster'] = db.labels_
        return group
    
    # Apply DBSCAN on each date cluster
    df = df.groupby('date_cluster').apply(apply_dbscan)
    
    df.reset_index(drop=True, inplace=True)
    
    # Create a unique 'event_id' for each unique combination of 'date_cluster' and 'regional_cluster'
    df['event_id'] = df.groupby(['date_cluster', 'regional_cluster']).ngroup()
    
    df = df.sort_values(by='event_id').reset_index(drop=True)
    
    return df

## 1.2 Load the function to shuffle and split data into training and testing according to event_id

In [14]:
def split_event_ids(event_id_pairs, test_size, random_seed):
    """
    Splits event_id pairs into training and test sets based on the specified test size.

    Args:
        event_id_pairs (list of tuples): List of event_id and their corresponding row counts.
        test_size (float): The proportion of the test set.
        random_seed (int): The seed for the random number generator.

    Returns:
        train_event_ids (list): List of event_ids for the training set.
        test_event_ids (list): List of event_ids for the test set.
    """
    np.random.seed(random_seed)
    np.random.shuffle(event_id_pairs)
    
    test_event_ids = []
    current_test_rows = 0
    total_rows = sum(count for _, count in event_id_pairs)
    test_rows_target = int(total_rows * test_size)
    
    for event_id, count in event_id_pairs:
        current_test_rows += count
        test_event_ids.append(event_id)
        if current_test_rows >= test_rows_target:
            break
    
    train_event_ids = [event_id for event_id, _ in event_id_pairs if event_id not in test_event_ids]
    
    return train_event_ids, test_event_ids

def shuffle_and_split(df, test_size=0.1, random_seed=42):
    """
    This function shuffles the DataFrame with respect to 'event_id' and then splits the data
    into training and test sets with approximately 10% test size with respect to 'event_id'.
    
    Args:
        df (pd.DataFrame): The input DataFrame
        test_size (float): The proportion of the test set
        random_seed (int): The seed for the random number generator
        
    Returns:
        X_train (pd.DataFrame): Training features
        X_test (pd.DataFrame): Test features
        y_train (pd.Series): Training labels
        y_test (pd.Series): Test labels
    """
    # Shuffle the dataframe based on event_id
    event_counts = df['event_id'].value_counts().sort_index()
    event_id_pairs = list(zip(event_counts.index, event_counts.values))
    
    train_event_ids, test_event_ids = split_event_ids(event_id_pairs, test_size, random_seed)
    
    X_train = df[df['event_id'].isin(train_event_ids)].drop(columns=['ignited'])
    y_train = df[df['event_id'].isin(train_event_ids)]['ignited']
    X_test = df[df['event_id'].isin(test_event_ids)].drop(columns=['ignited'])
    y_test = df[df['event_id'].isin(test_event_ids)]['ignited']
    
    return X_train, X_test, y_train, y_test



# Example usage:
# folder = '../../climada_petals/data/wildfire/outputs/'
# year = 2013
# file_path = os.path.join(folder, str(year), f'ignited_eu_{year}_gdf')
#     
# # Load the DataFrame from the file
# df = gpd.read_file(file_path)
# X_train, X_test, y_train, y_test = shuffle_and_split(df)


KeyboardInterrupt



## 1.3 Load the function for cross validation split according to specified column (e.g., 'event_id')

In [5]:
def custom_cv_split(X, y, n_splits=5, split_by_col='event_id', random_state=42):
    """
    This function creates cross-validation splits based on a specified column (e.g., 'event_id'),
    ensuring that all rows with the same value in the specified column are kept together in the same fold.

    Args:
        X (pd.DataFrame): Features
        y (pd.Series): Labels
        n_splits (int): Number of folds
        split_by_col (str): Column name to split by (e.g., 'event_id')
        random_state (int): Random seed for reproducibility

    Returns:
        splits (list): List of tuples containing train and validation indices for each fold
    """
    # Ensure reproducibility
    np.random.seed(random_state)
    
    # Get unique event IDs and shuffle them
    unique_event_ids = X[split_by_col].unique()
    np.random.shuffle(unique_event_ids)
    
    # Calculate the fold sizes
    fold_sizes = np.full(n_splits, X.shape[0] // n_splits, dtype=int)
    fold_sizes[: X.shape[0] % n_splits] += 1
    
    splits = []
    current = 0
    for fold_size in fold_sizes:
        val_event_ids = []
        current_fold_size = 0
        # Accumulate event IDs until the fold size limit is reached
        while current_fold_size < fold_size and current < len(unique_event_ids):
            event_id = unique_event_ids[current]
            event_size = len(X[X[split_by_col] == event_id])
            # Break if adding this event would exceed the fld size
            if current_fold_size + event_size > fold_size:
                break
            val_event_ids.append(event_id)
            current_fold_size += event_size
            current += 1
            
        # Get training event IDs by excluding validation event IDs
        train_event_ids = np.setdiff1d(unique_event_ids, val_event_ids)
        
        # Get the indices for training and validation sets
        train_indices = X[X[split_by_col].isin(train_event_ids)].index
        val_indices = X[X[split_by_col].isin(val_event_ids)].index
        
        # Append the indices for the current fold to the list
        splits.append((train_indices, val_indices))
        
    return splits


# 1.4 Load the function to train and evaluate ML models

In [6]:
def train_and_evaluate_models(df, random_state=42):
    df.dropna(inplace=True)

    # Split the data
    X_train_val, X_test, y_train_val, y_test = shuffle_and_split(df, test_size=0.1, random_seed=random_state)
    # Without resetting the index, index in custom_cv_split is not working and will get index-out-of-bound error
    X_train_val.reset_index(drop=True, inplace=True)
    X_test.reset_index(drop=True, inplace=True)
    y_train_val = y_train_val.reset_index(drop=True)
    y_test = y_test.reset_index(drop=True)
    
    X_train_val = X_train_val.drop(columns=['latitude', 'longitude', 'brightness', 'bright_t31', 'month', 'date', 'distance_km', 'confidence', 'geometry'])
    X_test = X_test.drop(columns=['latitude', 'longitude', 'brightness', 'bright_t31', 'month', 'date', 'distance_km', 'confidence', 'geometry'])

    # Initialize dictionaries to store the best models, hyperparameters, and scores
    classifier_names = ["LogisticRegression", "XGBClassifier"]
    best_models = {name: None for name in classifier_names}
    best_params = {name: None for name in classifier_names}
    best_scores = {name: 0 for name in classifier_names}

    # Define objective functions for Optuna
    def logisticregression_objective(trial):
        classifier_name = "LogisticRegression"
        logistic_c = trial.suggest_float('logistic_c', 1e-5, 1e5, log=True)
        classifier_obj = LogisticRegression(C=logistic_c, random_state=random_state)
        model_pipeline = make_pipeline(StandardScaler(), classifier_obj)
        cv_splits = custom_cv_split(X_train_val, y_train_val, n_splits=5, split_by_col='event_id', random_state=random_state)
        scores = cross_val_score(model_pipeline, X_train_val, y_train_val, cv=cv_splits, n_jobs=-1, scoring='roc_auc')
        score = scores.mean()
        if score > best_scores[classifier_name]:
            best_scores[classifier_name] = score
            best_params[classifier_name] = trial.params
            best_models[classifier_name] = classifier_obj # classifier_obj remain untrained
        return score

    def xgboost_objective(trial):
        classifier_name = "XGBClassifier"
        xgb_params = {
            'n_estimators': trial.suggest_int('n_estimators', 200, 2000),
            'max_depth': trial.suggest_int('max_depth', 3, 10),
            'learning_rate': trial.suggest_float('learning_rate', 1e-3, 1e-1, log=True),
            'subsample': trial.suggest_float('subsample', 0.2, 1.0),
            'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
            'gamma': trial.suggest_float('gamma', 0, 5),
            'reg_alpha': trial.suggest_float('reg_alpha', 0, 1),
            'reg_lambda': trial.suggest_float('reg_lambda', 0, 1),
            'random_state': random_state
        }
        classifier_obj = XGBClassifier(**xgb_params)
        cv_splits = custom_cv_split(X_train_val, y_train_val, n_splits=5, split_by_col='event_id', random_state=random_state)
        scores = cross_val_score(model_pipeline, X_train_val, y_train_val, cv=cv_splits, n_jobs=-1, scoring='roc_auc')
        score = scores.mean()
        if score > best_scores[classifier_name]:
            best_scores[classifier_name] = score
            best_params[classifier_name] = trial.params
            best_models[classifier_name] = classifier_obj # classifier_obj remain untrained
        return score

    # Optimize hyperparameters with Optuna
    study = optuna.create_study(sampler=optuna.samplers.TPESampler(), direction='maximize')
    study.optimize(logisticregression_objective, n_trials=20)
    study.optimize(xgboost_objective, n_trials=100)

    results = {}
    for name in classifier_names:
        if name == "LogisticRegression":
            best_models[name].set_params(max_iter=200)
            model_pipeline = make_pipeline(StandardScaler(), best_models[name])
        else:
            model_pipeline = best_models[name]

        model_pipeline.fit(X_train_val, y_train_val)
        train_accuracy = model_pipeline.score(X_train_val, y_train_val)
        test_accuracy = model_pipeline.score(X_test, y_test)
        y_pred = model_pipeline.predict(X_test)
        classification_rep = classification_report(y_test, y_pred)
        confusion_mat = confusion_matrix(y_test, y_pred)

        results[name] = {
            "train_roc_auc": train_accuracy,
            "test_roc_auc": test_accuracy,
            "classification_report": classification_rep,
            "confusion_matrix": confusion_mat
        }

        if name == "LogisticRegression":
            feature_importance = best_models[name].coef_[0]
        else:
            feature_importance = best_models[name].feature_importances_

        feature_importance_df = pd.DataFrame({'feature': X_train_val.columns, 'importance': feature_importance})
        feature_importance_df = feature_importance_df.sort_values(by='importance', ascending=False)
        results[name]["feature_importance"] = feature_importance_df

    return results



# 1.5 Load the function to train and evaluate ML models (Min 1s interval per API call)

In [ ]:
import pandas as pd
from geopy.geocoders import Nominatim

# Initialize Nominatim API
geolocator = Nominatim(user_agent="geoapiExercises")

def get_country(row):
    # Get the location from the latitude and longitude
    location = geolocator.reverse((row['latitude'], row['longitude']))
    address = location.raw['address']
    # Get the country
    country = address.get('country', '')
    return country

# 2. Max Consecutive days for each year

In [ ]:
folder = '../../climada_petals/data/wildfire/outputs/'
years = np.arange(2001, 2024)

In [ ]:
def find_max_consecutive_ones(nums) -> int:
    """
    This function takes in a list of binary values and returns the maximum number of consecutive ones.
    
    Args:
        nums (List[float]): The input list of float values
        
    Returns:
        int: The maximum number of consecutive ones
    """
    max_consecutive_ones = 0
    current_consecutive_ones = 0
    
    for num in nums:
        if num == 1.0:
            current_consecutive_ones += 1
        else:
            current_consecutive_ones = 0
        if current_consecutive_ones > max_consecutive_ones:
            max_consecutive_ones = current_consecutive_ones 
    return max_consecutive_ones

def max_consecutive_dates(group):
    # Remove duplicates dates
    group = group.drop_duplicates(subset='date')
    # Sort the DataFrame by date
    group = group.sort_values(by='date')
    # Calculate difference in days between consecutive days
    group['date_diff'] = group['date'].diff().dt.days.fillna(0)
    # Find the maximum number of consecutive days
    max_consecutive_days = find_max_consecutive_ones(group['date_diff'].values)
    return max_consecutive_days

In [ ]:
def select_region(df, bound):
    """
    Select the region of interest from the dataframe.
    df: hazard dataframe
    bound: tuple in format (lon_min, lat_min, lon_max, lat_max)
    """
    return df[(df['longitude'] >= bound[0]) & 
              (df['latitude'] >= bound[1]) & 
              (df['longitude'] <= bound[2]) & 
              (df['latitude'] <= bound[3])]

In [ ]:


max_consecutive_days_all_years = []
for year in years:
    # construct the file path
    file_path = os.path.join(folder, str(year), f'ignited_eu_{year}_gdf')
    
    # Load the DataFrame from the file
    df = gpd.read_file(file_path)
    
    geo_bound_uk = (-10, 49, 2, 61)
    
    df = select_region(df, geo_bound_uk)
    
    df = group_into_fire_events(df)
    
    # Calculate the maximum number of consecutive days
    max_consecutive_days = df.groupby('event_id').apply(max_consecutive_dates).max()
    max_consecutive_days_all_years.append(max_consecutive_days)

In [ ]:
# Plot the maximum number of consecutive days for each year
import matplotlib.pyplot as plt
plt.figure(figsize=(12, 6))
plt.hist(max_consecutive_days_all_years, color='skyblue', edgecolor='black', alpha=0.7)
plt.xlabel('Max Consecutive Days')
plt.ylabel('Frequency')
plt.title('Distribution of Maximum Consecutive Days (2001-2024)')
plt.grid(axis='y', linestyle='--', alpha=0.6)
plt.show()

# 3. Run ML model for each year

In [11]:
import os

folder = '../../climada_petals/data/wildfire/outputs/'

In [12]:
os.path.exists(folder)

True

In [ ]:
def preprocess_gdf(df):
    # drop rows without fwi values
    df.dropna(subset=['fwi'], inplace=True)
    # drop rows that the land type is sea
    df = df[df['land_cover'] != 200]
    
    # balance data
    df_true = df[df['ignited'] == True]
    df_false = df[df['ignited'] == False]
    
    df_false_downsampled = df_false.sample(n=min(df_true.shape[0], df_false.shape[0]), random_state=42, replace=False)
    df_true_downsampled = df_true.sample(n=min(df_true.shape[0], df_false.shape[0]), random_state=42, replace=False)
    
    df = pd.concat([df_true_downsampled, df_false_downsampled])
    
    # Preprocess the dataframe
    df = pd.get_dummies(df, columns=['land_cover'], prefix='land_cover')
    df['date'] = pd.to_datetime(df['date'])
    df['month'] = df['date'].dt.month
    df['month_sin'] = np.sin(2 * np.pi * df['month'] / 12)
    df['month_cos'] = np.cos(2 * np.pi * df['month'] / 12)
    
    return df
    

In [1]:
years = np.arange(2001, 2024)
geo_bound_uk = (-10, 49, 2, 61)

# Initialize an empty DataFrame to store the concatenated DataFrames
gdf_all_years = pd.DataFrame()
results = {}

for year in years:
    # construct the file path
    file_path = os.path.join(folder, str(year), f'ignited_eu_{year}_gdf')

    # Load the DataFrame from the file
    gdf = gpd.read_file(file_path)

    # Select the region of interest
    gdf = select_region(gdf, geo_bound_uk)

    # Concatenate the loaded DataFrame with the initial DataFrame
    gdf_all_years = pd.concat([gdf_all_years, gdf])

    # Group gdf into fire events
    gdf_all_years = group_into_fire_events(gdf_all_years)

    # Split data into training and testing sets and train and evaluate models
    result_one_year = train_and_evaluate_models(gdf_all_years)
    results[year] = result_one_year
    print(f"Finished training and evaluating models for {year}")


In [44]:
for year in years:
    for model, metrics in results[year].items():
        print(f"Results for {model}:")
        print("Training ROC AUC:", metrics['train_roc_auc'])
        print("Test ROC AUC:", metrics['test_roc_auc'])
        print("Classification Report:\n", metrics['classification_report'])
        print("Confusion Matrix:\n", metrics['confusion_matrix'])
        print("Feature Importance:\n", metrics['feature_importance'])
        print('------------------------------------')
    break

Results for LogisticRegression:
Training Accuracy: 0.7700218833679412
Test Accuracy: 0.7675706285786195
Classification Report:
               precision    recall  f1-score   support

       False       0.76      0.85      0.80      8629
        True       0.79      0.67      0.73      7264

    accuracy                           0.77     15893
   macro avg       0.77      0.76      0.76     15893
weighted avg       0.77      0.77      0.77     15893

Confusion Matrix:
 [[7297 1332]
 [2362 4902]]
Feature Importance:
            feature  importance
0              fwi    0.199789
4    land_cover_40    0.141254
5    land_cover_50    0.112411
9    land_cover_90    0.023947
19  land_cover_126    0.019786
2    land_cover_20    0.016416
18  land_cover_125    0.008038
12  land_cover_112    0.000000
3    land_cover_30   -0.000506
17  land_cover_124   -0.001221
15  land_cover_116   -0.015308
16  land_cover_121   -0.019716
10  land_cover_100   -0.021827
7    land_cover_70   -0.039546
1        elev

# 4. Visualization

In [ ]:
import seaborn as sns

In [ ]:
# Extracting data for plotting
data = []
for year in years:
    for model, metrics in results[year].items():
        data.append({'year': year, 'model': model, 'train_roc_auc': metrics['train_roc_auc'], 'test_roc_auc': metrics['test_roc_auc']})

# Creating dataframe
df = pd.DataFrame(data)

'''ROC AUC over years for LogisticRegression and XGBClassifier'''

# Plotting with y-axis range set from 0.85 to 1
plt.figure(figsize=(18, 9))

# Plot for LogisticRegression
plt.subplot(1, 2, 1)
sns.lineplot(data=df[df['model'] == 'LogisticRegression'], x='year', y='train_roc_auc', label='Train ROC AUC', marker='o')
sns.lineplot(data=df[df['model'] == 'LogisticRegression'], x='year', y='test_roc_auc', label='Test ROC AUC', marker='o')
for i in range(len(df[df['model'] == 'LogisticRegression'])):
    year = df[df['model'] == 'LogisticRegression'].iloc[i]['year']
    train_roc_auc = df[df['model'] == 'LogisticRegression'].iloc[i]['train_roc_auc']
    test_roc_auc = df[df['model'] == 'LogisticRegression'].iloc[i]['test_roc_auc']
    plt.text(year, train_roc_auc, f'{train_roc_auc:.2f}', fontsize=9, ha='right')
    plt.text(year, test_roc_auc, f'{test_roc_auc:.2f}', fontsize=9, ha='right')
plt.title('Logistic Regression ROC AUC over Years')
plt.xlabel('Year')
plt.ylabel('ROC AUC')
plt.ylim(0.85, 1)
plt.legend()
plt.xticks(years, rotation=45)
plt.grid(True, alpha=0.3)

# Plot for XGBClassifier
plt.subplot(1, 2, 2)
sns.lineplot(data=df[df['model'] == 'XGBClassifier'], x='year', y='train_roc_auc', label='Train ROC AUC', marker='o')
sns.lineplot(data=df[df['model'] == 'XGBClassifier'], x='year', y='test_roc_auc', label='Test ROC AUC', marker='o')
for i in range(len(df[df['model'] == 'XGBClassifier'])):
    year = df[df['model'] == 'XGBClassifier'].iloc[i]['year']
    train_roc_auc = df[df['model'] == 'XGBClassifier'].iloc[i]['train_roc_auc']
    test_roc_auc = df[df['model'] == 'XGBClassifier'].iloc[i]['test_roc_auc']
    plt.text(year, train_roc_auc, f'{train_roc_auc:.2f}', fontsize=9, ha='right')
    plt.text(year, test_roc_auc, f'{test_roc_auc:.2f}', fontsize=9, ha='right')
plt.title('XGBClassifier ROC AUC over Years')
plt.xlabel('Year')
plt.ylabel('ROC AUC')
plt.ylim(0.85, 1)
plt.legend()
plt.xticks(years, rotation=45)
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
'''Confusion Matrix'''

import numpy as np

# Summing confusion matrices for each model
logistic_conf_matrix = np.zeros((2, 2))
xgb_conf_matrix = np.zeros((2, 2))

for year in years:
    logistic_conf_matrix += np.array(results[year]['LogisticRegression']['confusion_matrix'])
    xgb_conf_matrix += np.array(results[year]['XGBClassifier']['confusion_matrix'])

# Plotting the confusion matrices
fig, axes = plt.subplots(1, 2, figsize=(18, 8))

# Logistic Regression Confusion Matrix
sns.heatmap(logistic_conf_matrix, annot=True, fmt='g', ax=axes[0], cmap='Blues')
axes[0].set_title('Logistic Regression Total Confusion Matrix (2001-2023)')
axes[0].set_xlabel('Predicted')
axes[0].set_ylabel('Actual')
axes[0].set_xticklabels(['Negative', 'Positive'])
axes[0].set_yticklabels(['Negative', 'Positive'])

# XGBClassifier Confusion Matrix
sns.heatmap(xgb_conf_matrix, annot=True, fmt='g', ax=axes[1], cmap='Blues')
axes[1].set_title('XGBClassifier Total Confusion Matrix (2001-2023)')
axes[1].set_xlabel('Predicted')
axes[1].set_ylabel('Actual')
axes[1].set_xticklabels(['Negative', 'Positive'])
axes[1].set_yticklabels(['Negative', 'Positive'])

plt.tight_layout()
plt.show()

In [ ]:
'''Feature Importance'''
# Aggregating feature importances for each model and getting absolute values
logistic_feature_importance = pd.DataFrame(columns=['feature', 'importance'])
xgb_feature_importance = pd.DataFrame(columns=['feature', 'importance'])

for year in years:
    logistic_feature_importance = pd.concat([logistic_feature_importance, results[year]['LogisticRegression']['feature_importance']], ignore_index=True)
    xgb_feature_importance = pd.concat([xgb_feature_importance, results[year]['XGBClassifier']['feature_importance']], ignore_index=True)

# Getting absolute values of importances
logistic_feature_importance['importance'] = logistic_feature_importance['importance'].abs()
xgb_feature_importance['importance'] = xgb_feature_importance['importance'].abs()

# Averaging feature importances
avg_logistic_feature_importance = logistic_feature_importance.groupby('feature').mean().reset_index()
avg_xgb_feature_importance = xgb_feature_importance.groupby('feature').mean().reset_index()

# Plotting
fig, axes = plt.subplots(1, 2, figsize=(18, 8))

# Logistic Regression Feature Importance
sns.barplot(data=avg_logistic_feature_importance, x='importance', y='feature', ax=axes[0], palette='Blues_d')
axes[0].set_title('Logistic Regression Average Feature Importance (2001-2023)')
axes[0].set_xlabel('Importance')
axes[0].set_ylabel('Feature')

# XGBClassifier Feature Importance
sns.barplot(data=avg_xgb_feature_importance, x='importance', y='feature', ax=axes[1], palette='Blues_d')
axes[1].set_title('XGBClassifier Average Feature Importance (2001-2023)')
axes[1].set_xlabel('Importance')
axes[1].set_ylabel('Feature')

plt.tight_layout()
plt.show()

# 5. Save the results

In [ ]:
import json
filename = 'results_all_years.json'
file_path = os.path.join(folder, filename)

# Write the results to a JSON file
with open(file_path, 'w') as f:
    json.dump(results, f)